In [ ]:
# import libraries
import keras
from utils.preprocess import *
from config import *

In [ ]:
# load dataset
dataset = load_unipen_dataset(no_cap)

# normalize dataset
def normalize(data, label):
    return tf.cast(data, tf.float32) / 255.0, label

if normalize_dataset:
    dataset = dataset.map(normalize)

dataset = dataset.shuffle(shuffle_buffer_size)

# split & filter dataset
train_size = int(train_prop * dataset.cardinality().numpy())
train_dataset = dataset.take(train_size).batch(batch_size)
test_dataset = dataset.skip(train_size).batch(batch_size)

In [ ]:
# build model
model = keras.Sequential([
    keras.layers.Conv2D(48, (3, 3), input_shape=(64, 64, 1)),
    keras.layers.ReLU(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(32, (3, 3)),
    keras.layers.ReLU(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.Flatten(),
    keras.layers.Dense(128 - 32),
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(train_dataset, epochs=epochs)

In [ ]:
# evaluate model
test_loss, test_acc = model.evaluate(test_dataset)

print()
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
# save model
target_model = "unipen_no_cap_model" if no_cap else "unipen_model"
model.save(f"data/{target_model}.h5")